In [ ]:
#bvh2smpl

import logging
import pickle
import sys
import numpy as np
import smplx
from bvh import Bvh
from scipy.spatial.transform import Rotation as R
import pdb
import math
 
# import bvh_tool, 
# import quat
 
names = [
    "Hips",
    "LeftUpLeg",
    "RightUpLeg",
    "Spine1",
    "LeftLeg",
    "RightLeg",
    "Spine2",
    "LeftFoot",#
    "RightFoot",
    "Spine3",
    "LeftFoot",#
    "RightFoot",
    "Neck",
    "LeftShoulder",
    "RightShoulder",
    "Head",
    "LeftArm",
    "RightArm",
    "LeftForeArm",
    "RightForeArm",
    "LeftHand",
    "RightHand",
]
def bvh_to_smplx(bvh_file, n_frames=None,scale_v=1):
    with open(bvh_file, 'r') as f:
        mocap = Bvh(f.read())
 
    if n_frames is None:
        num_frames = len(mocap.frames)
    else:
        num_frames = min(n_frames, len(mocap.frames))
 
    smplx_poses = np.zeros((num_frames, 24*3))
    smplx_trans = np.zeros((num_frames, 3))
 
    bvh_joint_names = set(mocap.get_joints_names())
 
    for joint_index, joint_name in enumerate(mocap.get_joints_names()):
        print(joint_name, joint_index)
 
    # 定义一个绕x转90°，再绕y转90°的旋转，用于校正BVH与SMPLX坐标系的差异（z朝前，x朝左，y朝上）
    rotation_correction = R.from_euler('XYZ', [90, 90, 0], degrees=True)
 
    for i in range(num_frames):
        print('Processing frame {}/{}'.format(i, num_frames), end='\r')
        for joint_index,joint_name in enumerate(names):
            if joint_name not in bvh_joint_names:
                logging.error(f'not joint_name:{joint_name}')
                exit(123)
 
            rotation = R.from_euler('YXZ', mocap.frame_joint_channels(i, joint_name, ['Yrotation','Xrotation','Zrotation' ]), degrees=True)
 
            # 仅对根关节（Hips）应用朝向校正
            if joint_name in ['Pelvis1','Hips']:
                # rotation = rotation * rotation_correction
                rotation = rotation_correction * rotation
 
            smplx_poses[i,  3*joint_index:3 * (joint_index + 1)] = rotation.as_rotvec()
 
            # 提取根关节平移
            if joint_name in ['Pelvis','Hips']:
                x, y, z = mocap.frame_joint_channels(i, joint_name, [ 'Xposition','Yposition', 'Zposition'])
                smplx_trans[i] = np.array([x,y, z])
 
    # 应用朝向校正
    # smplx_trans = rotation_correction_trans.apply(smplx_trans)
 
    # 反转Y轴平移方向
    # smplx_trans[:, 1] *= -1
 
    # 应用整体缩放
    smplx_trans /=scale_v
 
    return smplx_trans, smplx_poses
 
 
def save_npz(output_file, smplx_trans, smplx_poses, gender='neutral', model_type='smplx', frame_rate=30):
    np.savez(output_file, trans=smplx_trans, poses=smplx_poses, gender=gender, surface_model_type=model_type,
             mocap_frame_rate=frame_rate, betas=np.zeros(16))
 
 

bvh_file = "D:\\fsy\\project\\kdae\\F01A1V1.bvh"
output_file = "D:\\fsy\\project\\kdae\\F01A1V1.npz"
fps=30
scale_v=100.0
model_path=r"D:\fsy\project\smpl_model"
model = smplx.create(model_path=model_path, model_type="smpl", gender="FEMALE", batch_size=1)

parents = model.parents.detach().cpu().numpy()

# You can define betas like this.(default betas are 0 at all.)
rest = model(  # betas = torch.randn([1, num_betas], dtype=torch.float32)
)
rest_pose = rest.joints.detach().cpu().numpy().squeeze()[:24, :]

root_offset = rest_pose[0]
offsets = rest_pose - rest_pose[parents]
offsets[0] = root_offset
offsets *= scale_v

trans, rots_o = bvh_to_smplx(bvh_file, n_frames=600,scale_v=scale_v)

# rots = rots_o.reshape(rots_o.shape[0], -1, 3)
# print(rots_o.shape)
# print(rots.shape)
# rots = quat.from_axis_angle(rots)
N=rots_o.shape[0]
J=24
rots = rots_o.reshape(-1, 3)          # (N*J, 3)
quat = R.from_rotvec(rots).as_quat()  # 返回 (N*J, 4)  x,y,z,w
quat = np.array(quat).reshape(N, J, 4)
# print(quat.shape)

order = "yxz"  #rotation顺序
pos = offsets[None].repeat(N, axis=0)
positions = pos.copy()
positions[:, 0] = trans*scale_v
# rotations = np.degrees(quat.as_euler('yxz'))
rotations = R.from_quat(quat.reshape(-1,4)).as_euler(order, degrees=True).reshape(N, J, 3)
# print(rotations.shape)

bvh_data = {"rotations": rotations, "positions": positions, "offsets": offsets, "parents": parents, "names": names, "order": order, "frametime": 1 / fps}

# output = "0308.bvh"

# bvh_tool.save(output, bvh_data)

print('frame_rate: ', fps)
save_npz(output_file, trans, rots, frame_rate=20)

out = {"poses": rots_o, "trans": trans, "offsets": offsets, "parents": parents}
pickle.dump(out, open(f"test3.pkl", "wb"))

data包含的内容：
r_velocity 根旋转
l_velocity 根线速度
root_y 高度
ric_data 全局关节位置
rot_data 局部关节旋转
local_vel 局部线速度
feet_l
feet_r

In [ ]:
#smpl2humanml3d

import os
import numpy as np
import torch
from scipy.spatial.transform import Rotation as R
import joblib

# from common.skeleton import Skeleton
from common.quaternion import (
    qrot_np, qmul_np, qinv_np, qbetween_np, qfix, quaternion_to_cont6d_np
)
# from paramUtil import t2m_raw_offsets, t2m_kinematic_chain

# # 与 [motion_representation.ipynb](e:\szu\2025-11\motion_representation.ipynb) 保持一致的索引设置
# # Lower legs
# l_idx1, l_idx2 = 5, 8
# Right/Left foot
fid_r, fid_l = 8, 7
# # Face direction index: r_hip, l_hip, sdr_r, sdr_l
face_joint_indx = [2, 1, 17, 16]
# # l_hip, r_hip
# r_hip, l_hip = 2, 1

joints_num = 22

# T2M 骨架
skeleton = joblib.load("test3.pkl")
offsets = torch.from_numpy(skeleton["offsets"]) #(J,3)
parents = skeleton["parents"] #(J)

def forward_kinematics_np(quat, trans):
    """
    输入:
        quat: (T, J, 4) 四元数表示的关节旋转
        trans: (T, 3) 根关节平移
    输出:
        positions: (T, J, 3) 全局关节位置
    """
    T, J, _ = quat.shape
    positions = np.zeros((T, J, 3), dtype=np.float32)
    # 计算每个关节的全局位置
    for t in range(T):
        for j in range(J):
            if j == 0:
                positions[t, j] = trans[t]
            else:
                positions[t, j] = positions[t, parents[j]] + offsets[j]
    return positions

def aa_to_quat_wxyz(aa):
    """
    axis-angle -> quaternion (w,x,y,z)
    aa: (T, J, 3)
    """
    T, J, _ = aa.shape
    q_xyzw = R.from_rotvec(aa.reshape(-1, 3)).as_quat()  # (N,4) -> [x,y,z,w]
    q_xyzw = q_xyzw.reshape(T, J, 4)
    # 转为 (w,x,y,z)
    q_wxyz = np.concatenate([q_xyzw[..., 3:4], q_xyzw[..., 0:3]], axis=-1).astype(np.float32)
    return q_wxyz

def fk_smpl22_positions(trans, poses_axis_angle):
    """
    使用 T2M Skeleton 做前向运动学，得到全局关节位置。
    trans: (T,3) in meters
    poses_axis_angle: (T, 22*3) axis-angle
    return:
      positions: (T, 22, 3)
    """
    T = trans.shape[0]
    aa = poses_axis_angle.reshape(T, joints_num, 3).astype(np.float32)
    quat_params = aa_to_quat_wxyz(aa)  # (T,22,4)

    positions = forward_kinematics_np(quat_params, trans.astype(np.float32))  # (T,22,3)
    return positions

def foot_detect(positions, thres):
    """
    单点接触检测：每只脚只用一个关节的速度判定。
    """
    th2 = thres ** 2
    v_l = positions[1:, fid_l, :] - positions[:-1, fid_l, :]
    v_r = positions[1:, fid_r, :] - positions[:-1, fid_r, :]

    v2_l = np.sum(v_l ** 2, axis=-1, keepdims=True)  # (T-1,1)
    v2_r = np.sum(v_r ** 2, axis=-1, keepdims=True)  # (T-1,1)

    feet_l = (v2_l < th2).astype(np.float32)
    feet_r = (v2_r < th2).astype(np.float32)

    return feet_l, feet_r

def _safe_norm(v, eps=1e-8):
    n = np.linalg.norm(v, axis=-1, keepdims=True)
    return v / np.maximum(n, eps)

def _root_orientation_from_positions(frame_pos, face_joint_indx):
    # frame_pos： (J,3)
    # 用髋+肩的 across 定根坐标系：x=across, z=cross(up,x), y=cross(z,x)
    # x朝前，z朝右，y朝上
    r_hip, l_hip, sdr_r, sdr_l = face_joint_indx
    across = frame_pos[r_hip] - frame_pos[l_hip] + frame_pos[sdr_r] - frame_pos[sdr_l]
    x = _safe_norm(across)
    up = np.array([0.0, 1.0, 0.0], dtype=np.float32)
    z = _safe_norm(np.cross(up, x))
    y = _safe_norm(np.cross(z, x))
    Rm = np.stack([x, y, z], axis=1)  # 列为基
    q_xyzw = R.from_matrix(Rm).as_quat()  # x,y,z,w
    q_wxyz = np.concatenate([q_xyzw[3:4], q_xyzw[:3]], axis=0).astype(np.float32)
    return q_wxyz  # (4,)

def inverse_kinematics_np(positions, face_joint_indx, smooth_forward=False):
    """
    从全局关节位置 positions(T,J,3) 和静态 offsets(J,3)、parents(J) 恢复局部四元数 quat_local(T,J,4)
    思路：逐关节让 parent 坐标系下的 offset 向量旋到当前骨向量；转过去所需的最小旋转就是该关节的局部四元数
    """
    T, J, _ = positions.shape
    quat_local = np.zeros((T, J, 4), dtype=np.float32)
    quat_global = np.zeros((T, J, 4), dtype=np.float32)

    for t in range(T):
        root_quat = _root_orientation_from_positions(positions[t], face_joint_indx) #wxyz
        quat_local[t, 0] = root_quat
        quat_global[t, 0] = root_quat

        for j in range(1, J):
            parent = parents[j]
            parent_quat = quat_global[t, parent]
            parent_quat_inv = qinv_np(parent_quat[None, :])[0]

            bone_dir = positions[t, j] - positions[t, parent]  # (3,)
            bone_dir_local = qrot_np(parent_quat_inv[None, :], bone_dir[None, :])[0]
            offset = offsets[j].numpy()
            offset_norm = np.linalg.norm(offset)
            if offset_norm < 1e-8:
                quat_rel = np.array([1.0, 0.0, 0.0, 0.0], dtype=np.float32)
            else:
                target_dir = bone_dir_local / np.linalg.norm(bone_dir_local) * offset_norm
                quat_rel = qbetween_np(offset[None, :], target_dir[None, :])[0]  # (4,)
            quat_local[t, j] = quat_rel
            quat_global[t, j] = qmul_np(parent_quat[None, :], quat_rel[None, :])[0]

    return quat_local


def get_cont6d_params(positions):
    """
    复用 HumanML3D 的 IK + cont6d 生成方式：
      - IK 得到每帧关节四元数（smooth_forward=True）
      - 转连续6D
      - 根线速度/角速度
    """
    # skel = Skeleton(n_raw_offsets, kinematic_chain, "cpu")
    # (T, J, 4)
    # quat_params = skel.inverse_kinematics_np(positions, face_joint_indx, smooth_forward=True)
    quat_params = inverse_kinematics_np(positions)
    # 修正四元数
    quat_params = qfix(quat_params)

    cont_6d_params = quaternion_to_cont6d_np(quat_params)  # (T, J, 6)

    r_rot = quat_params[:, 0].copy()  # (T,4) root quat
    '''Root Linear Velocity'''
    velocity = (positions[1:, 0] - positions[:-1, 0]).copy()  # (T-1,3)
    velocity = qrot_np(r_rot[1:], velocity)
    '''Root Angular Velocity'''
    r_velocity = qmul_np(r_rot[1:], qinv_np(r_rot[:-1]))  # (T-1,4)

    return cont_6d_params, r_velocity, velocity, r_rot

def smplnpz_to_data(npz_path, out_vec_dir, out_joints_dir=None, feet_thre=0.002):
    """
    从 SMPL npz(trans, poses) 生成 HumanML3D data 并保存 .npy
    """
    os.makedirs(out_vec_dir, exist_ok=True)
    if out_joints_dir is not None:
        os.makedirs(out_joints_dir, exist_ok=True)

    npz = np.load(npz_path)
    trans = npz["trans"].astype(np.float32)           # (T,3)
    poses = npz["poses"].astype(np.float32)           # (T*24,3)
    T = trans.shape[0]
    poses = poses.reshape(T, 24, 3)[:, :22, :3]
    poses = poses.reshape(T, -1)                      # (T, 22*3) 轴角

    # 1) FK 得到原始全局关节位置
    positions = fk_smpl22_positions(trans, poses)     # (T,22,3)

    # 2) 放到地面
    floor_height = positions.min(axis=0).min(axis=0)[1]
    positions[:, :, 1] -= floor_height

    # 3) XZ 原点
    root_pos_init = positions[0]
    root_pose_init_xz = root_pos_init[0] * np.array([1, 0, 1], dtype=np.float32)
    positions = positions - root_pose_init_xz

    # # 4) 初始统一朝向 Z+
    # r_hip_i, l_hip_i, sdr_r_i, sdr_l_i = face_joint_indx
    # across1 = root_pos_init[r_hip_i] - root_pos_init[l_hip_i]
    # across2 = root_pos_init[sdr_r_i] - root_pos_init[sdr_l_i]
    # across = across1 + across2
    # across = across / np.linalg.norm(across, axis=-1, keepdims=True)
    # forward_init = np.cross(np.array([[0, 1, 0]], dtype=np.float32), across, axis=-1)
    # forward_init = forward_init / np.linalg.norm(forward_init, axis=-1, keepdims=True)
    # target = np.array([[0, 0, 1]], dtype=np.float32)
    # root_quat_init = qbetween_np(forward_init, target)
    # root_quat_init = np.ones(positions.shape[:-1] + (4,), dtype=np.float32) * root_quat_init
    # positions = qrot_np(root_quat_init, positions)

    # # 5) 作为 new ground truth positions
    global_positions = positions.copy()

    # 6) 脚接触
    feet_l, feet_r = foot_detect(positions, feet_thre)

    # 7) cont6d / root 速度 等
    cont_6d_params, r_velocity_q, velocity, r_rot = get_cont6d_params(positions)

    # 8) RIFKE（根局部化 + 用 r_rot 对齐所有帧到根朝向坐标系）
    def get_rifke(pos):
        pos = pos.copy()
        pos[..., 0] -= pos[:, 0:1, 0]
        pos[..., 2] -= pos[:, 0:1, 2]
        return qrot_np(np.repeat(r_rot[:, None], pos.shape[1], axis=1), pos)
    positions_rifke = get_rifke(positions)

    # 9) 构建 data 向量（与motion_representation.ipynb一致）
    root_y = positions[:, 0, 1:2]  # (T,1)
    r_velocity = np.arcsin(r_velocity_q[:, 2:3])      # (T-1,1) 取 y 轴分量
    l_velocity = velocity[:, [0, 2]]                  # (T-1,2)
    root_data = np.concatenate([r_velocity, l_velocity, root_y[:-1]], axis=-1)  # (T-1,4)

    rot_data = cont_6d_params[:, 1:].reshape(len(cont_6d_params), -1)           # (T, (J-1)*6)
    ric_data = positions_rifke[:, 1:].reshape(len(positions_rifke), -1)         # (T, (J-1)*3)

    local_vel = qrot_np(np.repeat(r_rot[:-1, None], global_positions.shape[1], axis=1),
                        global_positions[1:] - global_positions[:-1])
    local_vel = local_vel.reshape(len(local_vel), -1)                            # (T-1, J*3)

    data = root_data
    data = np.concatenate([data, ric_data[:-1]], axis=-1)
    data = np.concatenate([data, rot_data[:-1]], axis=-1)
    data = np.concatenate([data, local_vel], axis=-1)
    data = np.concatenate([data, feet_l, feet_r], axis=-1)                      # (T-1, ...)

    # 保存
    base = os.path.splitext(os.path.basename(npz_path))[0]
    out_vec_path = os.path.join(out_vec_dir, base + ".npy")
    np.save(out_vec_path, data)

    if out_joints_dir is not None:
        # 可选：保存用 recover_from_ric 可重建的 joints（用于自检）
        # 这里直接保存 positions（对齐后）
        out_joints_path = os.path.join(out_joints_dir, base + ".npy")
        np.save(out_joints_path, positions.astype(np.float32))

    print(f"[OK] {base}: data {data.shape}, positions {positions.shape}")
    return data, positions, global_positions, l_velocity

# 示例用法
if __name__ == "__main__":
    # 输入：由 BVH 转来的 SMPL npz（键：trans, poses）
    smpl_npz = r"D:\fsy\project\kdae\F01A1V1.npz"
    # 输出目录：data（HumanML3D/new_joint_vecs 风格）和可选 joints
    out_vec_dir = "./kdaee/new_joint_vecs/"
    out_joints_dir = "./kdaee/new_joints/"

    smplnpz_to_data(smpl_npz, out_vec_dir, out_joints_dir, feet_thre=0.002)
# ...existing code...